<a href="https://colab.research.google.com/github/NikoriakViktot/GeoHydroAI-V2/blob/main/geoembiding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install earthengine-api geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00


[Submitted on 29 Jul 2025 (v1), last revised 8 Sep 2025 (this version, v2)]

AlphaEarth Foundations: An embedding field model for accurate and efficient global mapping from sparse label data Christopher F. Brown, Michal R. Kazmierski, Valerie J. Pasquarella, William J. Rucklidge, Masha Samsikova, Chenhui Zhang, Evan Shelhamer, Estefania Lahera, Olivia Wiles, Simon Ilyushchenko, Noel Gorelick, Lihui Lydia Zhang, Sophia Alj, Emily Schechter, Sean Askay, Oliver Guinan, Rebecca Moore, Alexis Boukouvalas, Pushmeet Kohli Unprecedented volumes of Earth observation data are continually collected around the world, but high-quality labels remain scarce given the effort required to make physical measurements and observations. This has led to considerable investment in bespoke modeling efforts translating sparse labels into maps. Here we introduce AlphaEarth Foundations, an embedding field model yielding a highly general, geospatial representation that assimilates spatial, temporal, and measurement contexts across multiple sources, enabling accurate and efficient production of maps and monitoring systems from local to global scales. The embeddings generated by AlphaEarth Foundations are the only to consistently outperform a suite of other well-known/widely accepted featurization approaches tested on a diverse set of mapping evaluations without re-training. We have released a dataset of global, annual, analysis-ready embedding field layers from 2017 through 2024. Subjects: Computer Vision and Pattern Recognition (cs.CV); Machine Learning (cs.LG) Cite as: arXiv:2507.22291 [cs.CV] (or arXiv:2507.22291v2 [cs.CV] for this version)

https://doi.org/10.48550/arXiv.2507.22291 Focus to learn more Submission history

In [ ]:
import ee, geemap

# Авторизація через вікно Google (код підтвердження вставляти не треба — Colab сам обробить)
ee.Authenticate()

# Можеш явно вказати свій Project ID
ee.Initialize(project='ee-nikoriakviktor')



*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving basin_bc.geojson to basin_bc.geojson


In [ ]:
fname = next(iter(uploaded.keys()))
aoi = geemap.geojson_to_ee(fname).geometry()

In [ ]:
year = 2024
col = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
       .filterDate(f'{year}-01-01', f'{year+1}-01-01')
       .filterBounds(aoi))

img = col.mosaic().clip(aoi)  # <— зшиваємо тайли і кліпимо під басейн

# візуалізація
m = geemap.Map()
m.centerObject(aoi, 10)
m.add_layer(img, {'min':-0.3,'max':0.3,'bands':['A01','A06','A02']}, f'Emb {year}')
m


Map(center=[47.89792006888719, 24.93620332487922], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
def year_img(y):
    return (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
            .filterDate(f'{y}-01-01', f'{y+1}-01-01')
            .filterBounds(aoi)
            .mosaic()    # важливо: щоб покрити весь басейн
            .clip(aoi))

for y in range(2018, 2025):
    geemap.ee_export_image_to_drive(
        image=year_img(y),
        description=f'alphaearth_embeddings_{y}',
        folder='earth_engine',
        fileNamePrefix=f'embeddings_{y}',
        region=aoi,
        scale=10,
        # за потреби задай єдиний CRS для всього басейну:
        # crs='EPSG:32635',
        maxPixels=1e13
    )

In [20]:
for t in ee.batch.Task.list():
    s = t.status()
    desc = s.get('description')
    if desc and desc.startswith('alphaearth_embeddings_'):
        print(desc, '→', s.get('state'), s.get('error_message'))

alphaearth_embeddings_2024 → READY None
alphaearth_embeddings_2023 → READY None
alphaearth_embeddings_2022 → READY None
alphaearth_embeddings_2021 → RUNNING None
alphaearth_embeddings_2020 → RUNNING None
alphaearth_embeddings_2019 → COMPLETED None
alphaearth_embeddings_2018 → COMPLETED None
